In [91]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import scipy
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.preprocessing import MinMaxScaler

In [92]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [93]:
# # Hyper parameters
# num_classes = 5
# num_epochs = 10
# batch_size = 64
# # batch_size = 64
# learning_rate = 0.01

# segment_points = int(250 * (1667 / 2000)) # どれくらいで区切るか, ピークが1667点ごとくらいに現れる
# sequence_length = segment_points * 5 # 例 1セグメントあたりのサンプル数 250Hz * 5s
# overlap = int(sequence_length * 0.3)
# input_size = 1 # 特徴量の種類？
# hidden_size = 130 # 論文でいうhidden layerはこれ?
# num_layers = 2 # Two-layered lstm?

In [94]:
# Hyper parameters
# 追加ver.
num_classes = 5
num_epochs = 500
batch_size = 32
# batch_size = 64
learning_rate = 0.015

segment_points = int(250 * (1667 / 2000)) # どれくらいで区切るか, ピークが1667点ごとくらいに現れる
sequence_length = segment_points * 5 # 例 1セグメントあたりのサンプル数 250Hz * 5s
overlap = int(sequence_length * 0.25)
input_size = 1 # 特徴量の種類？
hidden_size = 130 # 論文でいうhidden layerはこれ?
num_layers = 2 # Two-layered lstm?

In [95]:
!pwd
!ls

/mnt/c/Users/grpro/workspace/grad_thesis/lstm_cnn_ensemble
CNN.ipynb   LSTM_colab.ipynb	 README.md  env
LSTM.ipynb  LSTM_tutorial.ipynb  data	    output.png


In [96]:
default_sample_rate = 2000
down_sample_rate = 250
ratio = default_sample_rate // down_sample_rate

In [97]:
# num_classes人分のdataframeを作る
radar_frame_list = []
scaler = MinMaxScaler((-1, 1)) # Min-Max Scaler -1~1

for i in range(1, num_classes + 1):
    wave_dem_2d = []
    file_path = "./data/radar_%02d.csv" % i
    radar_frame = pd.read_csv(file_path)
    wave = radar_frame.to_numpy().flatten()
    wave_dem = signal.decimate(wave, ratio)
    for i in range(len(wave_dem)):
      wave_dem_2d.append([wave_dem[i]])
    # wave_dem_ts = torch.from_numpy(wave_dem.astype(np.float32)).clone() # tensorにする
    print(i, len(wave), len(wave_dem_2d) // 900)

    remaining = len(wave_dem_2d)
    n = 0
    n_stop = sequence_length
    wave_segments = []

    while n_stop < len(wave_dem_2d):
        n_start = 0 + ((sequence_length - 1) - (overlap - 1)) * n
        n_stop = n_start + sequence_length
        tmp = []
        wave_segments.append(scaler.fit_transform(wave_dem_2d[n_start:n_stop]))
        n += 1
    
    radar_frame_list.append(wave_segments)

151899 1215199 168
155587 1244699 172
150337 1202699 167
150768 1206149 167
152524 1220199 169


In [98]:
radar_frame_list[0][0]

array([[-0.4311903 ],
       [-0.397682  ],
       [-0.36072063],
       ...,
       [-0.38272589],
       [-0.38216714],
       [-0.38215522]])

In [99]:
type(wave_dem_2d) # tensorにはしていない

list

In [100]:
wave_dem[0]

-2.3435893667948337e-05

In [101]:
print(radar_frame.to_numpy())

[[-2.37119380e-05]
 [-2.38183832e-05]
 [-2.39258871e-05]
 ...
 [-7.11852346e-06]
 [-6.73462954e-06]
 [-6.34069284e-06]]


In [102]:
labels = []
for i in range(len(radar_frame_list)):
    for j in range(len(radar_frame_list[i])):
        labels.append(i + 1)

labels_df = pd.Series(labels)
torch.tensor(labels_df)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [103]:
df = pd.DataFrame(radar_frame_list)
tmp = df.to_numpy().flatten()
df = pd.Series(tmp).dropna()
tmp = df.to_numpy().flatten()

tmp_labels = labels_df.to_numpy().flatten()
for i in reversed(range(len(tmp))):
  if len(tmp[i]) != sequence_length:
    print(i, len(tmp[i]))
    tmp = np.delete(tmp, i)
    tmp_labels = np.delete(tmp_labels, i)

df = pd.Series(tmp)
labels_df = pd.Series(tmp_labels)

976 425
780 1009
587 578
394 368
194 580


In [104]:
df, type(df), type(df[0]), type(df[0][0]), type(df[0][0][0]), len(df[0])

(0      [[-0.43119030027867716], [-0.39768199524563536...
 1      [[-0.3534415565242706], [-0.35845964800756086]...
 2      [[-0.18042866945988206], [-0.19453438296124625...
 3      [[-0.6546532362142365], [-0.621911348400261], ...
 4      [[0.6066640776486409], [0.636202403689861], [0...
                              ...                        
 967    [[0.2978653744735571], [0.2835063588966514], [...
 968    [[-0.283224584578006], [-0.3079258528454299], ...
 969    [[0.840542403153628], [0.8258411435483969], [0...
 970    [[0.6445997743981384], [0.6645212646281974], [...
 971    [[-0.6693811764847345], [-0.6616021006824988],...
 Length: 972, dtype: object,
 pandas.core.series.Series,
 numpy.ndarray,
 numpy.ndarray,
 numpy.float64,
 1040)

In [105]:
labels_df

0      1
1      1
2      1
3      1
4      1
      ..
967    5
968    5
969    5
970    5
971    5
Length: 972, dtype: int64

In [106]:
class MyDataset(Dataset):
    def __init__(self, dataset, labels, root_dir, transform=None) -> None:
        # super().__init__()
        self.radar_heartbeat = dataset
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx = idx.tolist()
        # heartbeat_segment = torch.FloatTensor(self.radar_heartbeat[idx])
        # subject_label = torch.LongTensor(self.labels[idx])
        # if self.transform: # transform適用されず?
        # heartbeat_segment = self.transform(self.radar_heartbeat[idx]) # tensorに
        onehot_label = torch.eye(num_classes)[self.labels[idx] - 1] # one hot encording
        # onehot_label = self.transform(self.labels[idx]) # tensorに

        # return heartbeat_segment, subject_label
        # return torch.from_numpy(self.radar_heartbeat.to_numpy().copy()), torch.tensor(labels_df)    
        # return torch.stack(self.radar_heartbeat.to_numpy().tolist()), torch.tensor(labels_df)    
        # return self.radar_heartbeat[idx], labels_df[idx]
        return torch.tensor(self.radar_heartbeat[idx]), onehot_label



    def __len__(self):
        return len(self.radar_heartbeat)

In [107]:
dataset = MyDataset(df, labels_df, "./data/", transform=transforms.ToTensor())

In [108]:
seg, lab = next(iter(dataset))

In [109]:
type(seg), len(seg), lab

(torch.Tensor, 1040, tensor([1., 0., 0., 0., 0.]))

In [110]:
train_size = int(0.8 * len(df.values))
test_size = len(df.values) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
print(f"full: {len(dataset)} -> train: {len(train_set)}, test: {len(test_set)}")

full: 972 -> train: 777, test: 195


In [111]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [112]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    # print(x.size(), h0.size(), c0.size())
    # out, _ = self.rnn(x, h0)
    out, _ = self.lstm(x, (h0, c0))

    out = out[:, -1, :]

    out = self.fc(out)

    return out

In [113]:
model = LSTM(input_size, hidden_size, num_layers, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [114]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (signals, labels) in enumerate(train_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    labels = labels.to(device)

    # print(signals.size())
    outputs = model(signals)
    # print(outputs)
    loss = criterion(outputs, labels) # will check the shapes of outputs and labels

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # if (i + 1) % 15 == 0:
  print(f'Epoch [{epoch+1}/`{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

/tmp/ipykernel_837/3942043653.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(signals)


In [ ]:
print(i+1)

19


In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  softmax = nn.Softmax()
  for i, (signals, one_hot_labels) in enumerate(test_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    one_hot_labels = one_hot_labels.to(device)
    # print(len(one_hot_labels))
    outputs = model(signals)
    for j, out in enumerate(outputs):
      outputs[j] = softmax(out)

    _, predicted = torch.max(outputs.data, 1) # predicted per batch size

    n_samples += one_hot_labels.size(0) # add batch_size
    # labels = [] # labels per batch size
    for k, labels in enumerate(one_hot_labels):
      # labels.append(torch.argmax(one_hot_labels[k]))
      if predicted[k] == torch.argmax(labels):
        # print(predicted[k], torch.argmax(labels[k]))
        n_correct += 1

    acc = 100.0 * n_correct / n_samples
    print(f'{n_correct} / {n_samples} = Acc: {acc} %')
    # print(f'Accuracy of the network on the 10000 test images: {acc} %')

15 / 32 = Acc: 46.875 %
30 / 64 = Acc: 46.875 %
42 / 96 = Acc: 43.75 %
52 / 128 = Acc: 40.625 %
55 / 145 = Acc: 37.93103448275862 %


/tmp/ipykernel_837/3918647096.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(signals)
/tmp/ipykernel_837/3918647096.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs[j] = softmax(out)


In [ ]:
predicted.size()

torch.Size([17])

In [ ]:
torch.argmax(labels)
labels.size()

torch.Size([5])

In [ ]:
predicted, (labels)

(tensor([0, 1, 2, 3, 0, 0, 3, 1, 1, 2, 2, 1, 1, 2, 0, 0, 0], device='cuda:0'),
 tensor([0., 0., 0., 0., 1.], device='cuda:0'))

In [ ]:
result = (predicted == torch.argmax(labels[len(labels)-1]))
result, labels[len(labels)-1]

(tensor([ True, False, False, False,  True,  True, False, False, False, False,
         False, False, False, False,  True,  True,  True], device='cuda:0'),
 tensor(1., device='cuda:0'))